In [1]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [2]:
df = pd.read_csv("../data/adult.csv")
df = df.replace("?", np.nan)
df = df.dropna()
df

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,32556,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,38,United-States,<=50K,36
32557,32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K,173
32558,32558,58,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,40,United-States,<=50K,40
32559,32559,22,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,20,United-States,<=50K,38


# Task 1
Print the count of men and women in the dataset.

In [3]:
men = df["sex"].tolist().count("Male")
women = df["sex"].tolist().count("Female")
print(f"Men = {men}, Women = {women}")

Men = 20380, Women = 9782


# Task 2
Find the average age of men in dataset

In [5]:
average_age_men = df.query('sex=="Male"')["age"].mean()
print(average_age_men)

39.18400392541707


# Task 3
Get the percentage of people from Poland (native-country)

In [8]:
from_poland = df["native-country"].tolist().count("Poland") / df["native-country"].count() * 100
print(f"The percentage of people from Poland - {from_poland.round(3)} %")

The percentage of people from Poland - 0.186 %


# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [9]:
age_more_than_50k = df.query('salary==">50K"')["age"]
age_less_than_50k = df.query('salary=="<=50K"')["age"]
more_mean = age_more_than_50k.mean()
less_mean = age_less_than_50k.mean()
more_standard_deviation = age_more_than_50k.std()
less_standard_deviation =  age_less_than_50k.std()
print(f"The average age of people who earn >50K $ per year - {more_mean}")
print(f"The average age of people who earn <=50K $ per year - {less_mean}")
print(f"The age standard deviation of people who earn >50K $ per year - {more_standard_deviation}")
print(f"The age standard deviation of people who earn <=50K $ per year - {less_standard_deviation}")

The average age of people who earn >50K $ per year - 43.95911028236548
The average age of people who earn <=50K $ per year - 36.60806038668668
The age standard deviation of people who earn >50K $ per year - 10.269632835673852
The age standard deviation of people who earn <=50K $ per year - 13.464631257161633


# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [10]:
education = ["HS-grad", "11th", "9th", "Some-college", "7th-8th", "5th-6th", "10th", "12th", "1st-4th", "Preschool"]
without_higher_education = df[df["education"].isin(education)]
with_50k_salary = without_higher_education.query('salary==">50K"')
with_50k_salary

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
7,7,52,Self-emp-not-inc,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,45,United-States,>50K,307
10,10,37,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,Black,Male,80,United-States,>50K,116
55,55,43,Private,Some-college,Married-civ-spouse,Tech-support,Husband,White,Male,40,United-States,>50K,341
67,67,53,Private,HS-grad,Married-civ-spouse,Adm-clerical,Wife,White,Female,40,United-States,>50K,225
68,68,49,Self-emp-inc,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,50,United-States,>50K,194
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32462,32462,48,Self-emp-inc,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,50,United-States,>50K,343
32518,32518,57,Local-gov,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,>50K,116
32519,32519,46,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,48,United-States,>50K,239
32557,32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K,173


# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [16]:
education_stat = df.groupby("education")["age"].describe()

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
10th,820.0,37.897561,16.225795,17.0,23.0,36.0,52.0,90.0
11th,1048.0,32.363550,15.089307,17.0,18.0,28.5,43.0,90.0
12th,377.0,32.013263,14.373710,17.0,19.0,28.0,41.0,79.0
1st-4th,151.0,44.622517,14.929051,19.0,33.0,44.0,56.0,81.0
5th-6th,288.0,41.649306,14.754622,17.0,28.0,41.0,53.0,82.0
7th-8th,557.0,47.631957,15.737479,17.0,34.0,49.0,60.0,90.0
9th,455.0,40.303297,15.335754,17.0,28.0,38.0,53.0,90.0
Assoc-acdm,1008.0,37.286706,10.509755,19.0,29.0,36.0,44.0,90.0
Assoc-voc,1307.0,38.246366,11.181253,19.0,30.0,37.0,45.0,84.0


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [85]:
x = df.query('sex=="Male"')[["marital-status", "salary"]]
married = ["Married-civ-spouse", "Married-spouse-absent", "Married-AF-spouse"]
non_married = ["Never-married", "Divorced", "Separated", "Widowed"]
mar = x[x["marital-status"].isin(married)].groupby("salary").count()
n_mar = x[x["marital-status"].isin(non_married)].groupby("salary").count()
mar

,marital-status
salary,
<=50K,7052
>50K,5723


# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [14]:
max_hours = df["hours-per-week"].max()
work_time_stat = df["hours-per-week"].value_counts()[max_hours]
print(f"Max hours per week - {max_hours}")
print(f"Count of people working the same count of hours - {work_time_stat}")

Max hours per week - 99
Count of people working the same count of hours - 78


# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [106]:
x = df.corr()
x

,Unnamed: 0,age,hours-per-week,salary K$
Unnamed: 0,1.000000,-0.001126,-0.001890,0.000129
age,-0.001126,1.000000,0.101599,0.208203
hours-per-week,-0.001890,0.101599,1.000000,0.196378
salary K$,0.000129,0.208203,0.196378,1.000000
